In [35]:
import pandas as pd
import numpy as np
import requests
import json
from scipy.optimize import linprog

# --- Global Configuration ---
# Your FPL Team ID (REPLACE THIS WITH YOUR ACTUAL ID)
TEAM_ID = 177230 

# The target Gameweek for prediction. This will be automatically advanced 
# if it's currently set to a completed GW.
GW_TEST = 8 

# --- API Endpoints ---
# We use the FPL classic endpoints for data fetching
API_ENDPOINTS = {
    "bootstrap_static": "https://fantasy.premierleague.com/api/bootstrap-static/",
    "fixtures": "https://fantasy.premierleague.com/api/fixtures/",
    "entry_summary": f"https://fantasy.premierleague.com/api/entry/{TEAM_ID}/",
    "picks": f"https://fantasy.premierleague.com/api/entry/{{TEAM_ID}}/event/{{GW}}/picks/",
    # --- FIX: Ensure 'event_live' is correctly defined for actual scores
    "event_live": "https://fantasy.premierleague.com/api/event/{GW}/live/",
    "history_player": "https://fantasy.premierleague.com/api/element-summary/{ID}/"
}

# --- Global Variables initialized in Cell 2 ---
# Will be populated after data fetching
LATEST_GW_PLAYED = 0
df_all_players = pd.DataFrame()
teams_df = pd.DataFrame()
fixture_df = pd.DataFrame()
player_map = {}

# User-specific data (fetched from entry_summary/picks)
SIM_BANK = 1000  # Default £100.0m
SIM_FT = 1       # Default 1 free transfer
simulated_current_team_list = []

# Map element_type IDs to position names
pos_map = {1: 'GKP', 2: 'DEF', 3: 'MID', 4: 'FWD'}
pos_map_id_name = {'GKP': 1, 'DEF': 2, 'MID': 3, 'FWD': 4}

# --- Global Variables initialized in Cell 3 ---
historical_results = []
test_df = pd.DataFrame()
rf_prediction_summary = {}

# --- Global Variables initialized in Cell 7 ---
xgb_prediction_summary = {}

# --- Helper function used across cells (defined here for re-run safety) ---
def fetch_data(url):
    """Fetches data from a given FPL API URL."""
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        # We print the URL and error for debugging
        print(f"Error fetching data from {url}: {e}")
        return None


In [36]:
import pandas as pd
import numpy as np
import requests # Assuming imports from Cell 1 are available

# --- Global Configuration Dependencies (Assumed to be defined in Cell 1) ---
# GW_TEST, LATEST_GW_PLAYED, API_ENDPOINTS, pos_map, fetch_data, SIM_BANK, SIM_FT, simulated_current_team_list, df_all_players, fixture_df, player_map

# Ensure global variables are accessible for modification in this cell
global LATEST_GW_PLAYED, GW_TEST, SIM_BANK, SIM_FT, simulated_current_team_list
global df_all_players, teams_df, fixture_df, player_map


def get_base_player_data(static_data):
    """Extracts and preprocesses base player data."""
    
    global player_map, teams_df
    
    elements_df = pd.DataFrame(static_data['elements'])
    teams_df = pd.DataFrame(static_data['teams'])
    
    # 1. Select key player attributes
    player_data = elements_df[[
        'id', 'web_name', 'element_type', 'team', 'now_cost', 'selected_by_percent', 
        'form', 'value_season', 'total_points', 'transfers_in', 'transfers_out'
    ]].copy()
    
    # 2. Rename and map columns
    player_data = player_data.rename(columns={
        'element_type': 'position_id',
        'team': 'team_id',
        'now_cost': 'price_100k'
    })
    player_data['position'] = player_data['position_id'].map(pos_map)
    
    # 3. Add Team Name
    team_name_map = teams_df.set_index('id')['name'].to_dict()
    player_data['team_name'] = player_data['team_id'].map(team_name_map)
    
    # 4. Convert string metrics to numeric (important for model)
    for col in ['selected_by_percent', 'form', 'value_season']:
        # Coerce errors allows non-numeric values to become NaN
        player_data[col] = pd.to_numeric(player_data[col], errors='coerce')
        
    # Create the global map for player ID to web_name lookup
    player_map = player_data.set_index('id')['web_name'].to_dict()
    
    return player_data, teams_df

def process_fixtures(fixtures_data, player_data):
    """
    Processes fixture data to calculate Future Fixture Difficulty (FFD) 
    for each team based on the next 5 Gameweeks starting at GW_TEST.
    (FIXED to use team_h_difficulty and team_a_difficulty)
    """
    global fixture_df
    
    fixtures_df = pd.DataFrame(fixtures_data)
    
    # Filter for future/current Gameweeks (using GW_TEST)
    fixtures_df = fixtures_df[fixtures_df['event'] >= GW_TEST].copy()

    # --- Fixture Difficulty Handling ---
    required_cols = ['team_h_difficulty', 'team_a_difficulty']
    if not all(col in fixtures_df.columns for col in required_cols):
        print(f"WARNING: Required difficulty columns {required_cols} missing in future fixture data. Fixture Difficulty feature will be 3.0 (Neutral).")
        player_data['fixture_difficulty'] = 3.0
        fixture_df = fixtures_df
        return player_data

    # Apply the FFD calculation to all players based on their team_id
    def calculate_fixture_difficulty(fixtures, team_id, current_gw, num_gws=5):
        """Calculates the average fixture difficulty for a team over the next N gameweeks."""
        # 1. Filter for the team's upcoming fixtures
        team_fixtures = fixtures[
            (fixtures['team_h'] == team_id) | (fixtures['team_a'] == team_id)
        ].copy()
        
        team_fixtures = team_fixtures[
            (team_fixtures['event'] >= current_gw) & 
            (team_fixtures['event'] < current_gw + num_gws)
        ]
        
        # 2. Determine the correct difficulty column for each fixture using numpy
        is_home = team_fixtures['team_h'] == team_id
        
        difficulties = np.where(
            is_home, 
            team_fixtures['team_h_difficulty'], 
            team_fixtures['team_a_difficulty']
        ).tolist()

        if not difficulties:
            # If a team has no future fixtures (e.g., end of season), default to neutral
            return 3.0
        
        return np.mean(difficulties)

    ffds = player_data['team_id'].apply(lambda x: calculate_fixture_difficulty(fixtures_df, x, GW_TEST))
    player_data['fixture_difficulty'] = ffds
    
    fixture_df = fixtures_df
    
    return player_data

def get_user_data(summary_data):
    """
    Extracts user-specific data (bank, FTs, current team) 
    from LATEST_GW_PLAYED (the last completed GW).
    """
    global SIM_BANK, SIM_FT, simulated_current_team_list
    
    # Target GW for fetching picks must be the LATEST_GW_PLAYED, not GW_TEST
    picks_target_gw = LATEST_GW_PLAYED 
    
    if picks_target_gw < 1:
        print("Cannot fetch user picks: No Gameweek completed yet.")
        simulated_current_team_list = [] # Default empty
        return
        
    # Attempt to get the latest Bank value (FPL stores it in 100k units)
    # Use .get() with a default in case the key is missing
    SIM_BANK = summary_data.get('last_deadline_bank', 1000)
    # Assume 1 FT unless history provides better data
    SIM_FT = summary_data.get('event_transfers', 1)
    
    picks_url = API_ENDPOINTS["picks"].replace("{GW}", str(picks_target_gw)).replace("{TEAM_ID}", str(TEAM_ID))
    picks_data = fetch_data(picks_url)
    
    if picks_data and 'picks' in picks_data:
        # 'element' is the player ID
        simulated_current_team_list = [p['element'] for p in picks_data['picks']]
        
        print(f"\nUser Team Data (Fetched from Completed GW {picks_target_gw}):")
        print(f"  Bank: £{SIM_BANK / 10:.1f}m (in 100k units: {SIM_BANK})")
        print(f"  Free Transfers (Assumed): {SIM_FT}")
        print(f"  Current Squad Size: {len(simulated_current_team_list)}")
    else:
        print(f"Error: Could not retrieve current squad picks for GW {picks_target_gw}. Using empty squad.")
        simulated_current_team_list = []


# ----------------------------------------------------------------------
# --- EXECUTION ---
# ----------------------------------------------------------------------

print("--- Cell 2: Fetching Data ---")

# 1. Fetch Static Data (Players, Teams)
static_data = fetch_data(API_ENDPOINTS["bootstrap_static"])
if static_data is None:
    raise RuntimeError("Failed to fetch bootstrap-static data. Cannot continue.")

# 2. Determine LATEST_GW_PLAYED and set GW_TEST
events_df = pd.DataFrame(static_data['events'])

# Get the highest ID for a finished event
LATEST_GW_PLAYED = events_df[events_df['finished'] == True]['id'].max()

if pd.isna(LATEST_GW_PLAYED) or LATEST_GW_PLAYED < 1:
    LATEST_GW_PLAYED = 0
    print("WARNING: No gameweeks appear to be finished. Backtesting will be skipped.")
else:
    LATEST_GW_PLAYED = int(LATEST_GW_PLAYED)

# Adjust GW_TEST if it's outdated or equals the latest completed one
if GW_TEST <= LATEST_GW_PLAYED:
    # Set the target GW to the next one
    GW_TEST = LATEST_GW_PLAYED + 1
    
print(f"Latest completed Gameweek (LATEST_GW_PLAYED): {LATEST_GW_PLAYED}")
print(f"Target Prediction Gameweek (GW_TEST): {GW_TEST}")

# 3. Process Base Player Data and Fixtures
df_base_players, teams_df = get_base_player_data(static_data)
# Fixture data is fetched here
df_all_players = process_fixtures(fetch_data(API_ENDPOINTS["fixtures"]), df_base_players)

# 4. Fetch User's Summary Data for Bank/FTs/Squad
user_summary_data = fetch_data(API_ENDPOINTS["entry_summary"].replace("{TEAM_ID}", str(TEAM_ID)))
if user_summary_data is None:
    raise RuntimeError("Failed to fetch user summary data.")
    
get_user_data(user_summary_data)

print("Data fetching and preprocessing complete.")


--- Cell 2: Fetching Data ---
Latest completed Gameweek (LATEST_GW_PLAYED): 7
Target Prediction Gameweek (GW_TEST): 8

User Team Data (Fetched from Completed GW 7):
  Bank: £0.5m (in 100k units: 5)
  Free Transfers (Assumed): 1
  Current Squad Size: 15
Data fetching and preprocessing complete.


In [37]:
import pandas as pd
import numpy as np
import requests
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# --- Global Data Structures for Backtesting Results ---
# (Assumes initialization in Cell 1)
global historical_results, test_df
historical_results = []
test_df = pd.DataFrame() # To hold the final GW_TEST predictions

# --- Feature Engineering & Data Preparation ---
def prepare_data_for_gw(target_gw, base_players_df):
    """
    Fetches the actual total points scored by players in GW 'target_gw'
    and combines it with the base player features for model training.
    """
    
    if target_gw < 1:
        return pd.DataFrame()

    print(f"  -> Gathering actual scores for GW {target_gw}...")
    
    try:
        # 1. Fetch live event data for the target Gameweek (where actual scores are recorded)
        gw_url = API_ENDPOINTS["event_live"].replace("{GW}", str(target_gw))
        gw_data = fetch_data(gw_url)
        
        if gw_data is None:
            # fetch_data prints the error
            return pd.DataFrame()
            
        # 2. Extract actual scores for this GW
        gw_scores = {}
        for element in gw_data.get('elements', []):
            gw_scores[element['id']] = element['stats'].get('total_points', 0)
        
        # 3. Create training snapshot: Use current features but map the target score
        temp_df = base_players_df.copy()
        temp_df['target_points'] = temp_df['id'].map(gw_scores)
        temp_df = temp_df.dropna(subset=['target_points'])
        
        # Filter out players who scored 0 (assuming they mostly didn't play or were injured)
        temp_df = temp_df[temp_df['target_points'] > 0]
        
        temp_df['gw_train'] = target_gw
        
        return temp_df
        
    except Exception as e:
        print(f"  -> Skipping GW {target_gw} due to data error: {e}")
        return pd.DataFrame()


def train_and_predict(train_data, predict_data):
    """Trains the Random Forest model and returns predictions."""
    
    FEATURES = [
        'price_100k', 
        'position_id', 
        'selected_by_percent', 
        'form', 
        'value_season', 
        'team_id',
        'fixture_difficulty',
    ]
    TARGET = 'target_points'
    
    # 1. Clean Data for Training
    train_data = train_data.copy().dropna(subset=FEATURES + [TARGET])
    
    if train_data.empty or len(train_data) < 50:
        print("Model training skipped: Training data is empty or insufficient.")
        # Fallback prediction using current form if model training fails
        predict_data['predicted_points_rf'] = predict_data['form'].astype(float) * 0.8
        return predict_data.copy()
        
    X_train = train_data[FEATURES]
    y_train = train_data[TARGET]
    X_test = predict_data[FEATURES]
    
    # 2. Train Model
    model = RandomForestRegressor(n_estimators=100, 
                                  random_state=42, 
                                  max_depth=8, 
                                  min_samples_split=5, 
                                  n_jobs=-1)
    
    model.fit(X_train, y_train)
    
    # 3. Predict
    predictions = model.predict(X_test)
    predict_data['predicted_points_rf'] = np.round(predictions, 2)
    
    return predict_data


# ----------------------------------------------------------------------
# --- ITERATIVE BACKTESTING EXECUTION ---
# ----------------------------------------------------------------------

print("--- Cell 3: Running Random Forest Backtesting & Prediction ---")

# 1. Prepare base data for prediction (GW_TEST)
test_df_base = df_all_players.copy()
test_df_base['target_points'] = np.nan # Target is unknown for GW_TEST

# Set the maximum GW for the backtesting loop
max_backtest_gw = LATEST_GW_PLAYED

# List to hold all training data (for the final prediction)
full_training_data_list = []

if max_backtest_gw < 1:
    print("Skipping backtesting: No completed Gameweeks found.")

# 2. Backtesting Loop (GW 1 up to LATEST_GW_PLAYED)
for gw_id in range(1, max_backtest_gw + 1):
    
    # 2a. Assemble training data: Use all historical GW scores *before* the current gw_id
    current_gw_training_data_list = []
    for train_gw in range(1, gw_id):
        train_gw_data = prepare_data_for_gw(train_gw, df_all_players)
        if not train_gw_data.empty:
            current_gw_training_data_list.append(train_gw_data)
            
    # 2b. Assemble prediction data (the GW itself, with actual results)
    predict_gw_df = prepare_data_for_gw(gw_id, df_all_players)
    
    if predict_gw_df.empty:
        print(f"\n[BACKTESTING GW {gw_id}] Could not fetch actual results. Skipping.")
        continue
        
    # --- Only proceed with training/prediction if we have history ---
    if current_gw_training_data_list:
        train_data_hist = pd.concat(current_gw_training_data_list, ignore_index=True)
    
        # 2c. Train and Predict
        print(f"\n[BACKTESTING GW {gw_id}] Training model on {len(train_data_hist)} samples. Predicting GW {gw_id}...")
        # Use a copy of predict_gw_df for prediction output
        predicted_gw_df = train_and_predict(train_data_hist, predict_gw_df.copy()) 
        
        # 2d. Record Results
        if 'predicted_points_rf' in predicted_gw_df.columns and not predicted_gw_df.empty:
            
            # Find the actual top scorer
            actual_top_scorer = predicted_gw_df.sort_values(by='target_points', ascending=False).iloc[0]
            
            # Find the predicted top scorer
            predicted_top_scorer = predicted_gw_df.sort_values(by='predicted_points_rf', ascending=False).iloc[0]
            
            # Extract the actual points for the player the model predicted would be top scorer
            predicted_player_actual_pts = predicted_top_scorer['target_points']
            
            historical_results.append({
                'gw': gw_id,
                'actual_top_player': actual_top_scorer['web_name'],
                'actual_points': actual_top_scorer['target_points'],
                'predicted_top_player': predicted_top_scorer['web_name'],
                'predicted_points': predicted_top_scorer['predicted_points_rf'],
                'mae': mean_absolute_error(predicted_gw_df['target_points'], predicted_gw_df['predicted_points_rf'])
            })
            
            print(f"  Actual Top Scorer: {actual_top_scorer['web_name']} ({actual_top_scorer['target_points']} Pts)")
            print(f"  Predicted Top Scorer: {predicted_top_scorer['web_name']} (Predicted: {predicted_top_scorer['predicted_points_rf']:.2f} Pts, Actual: {predicted_player_actual_pts} Pts)")
    else:
        print(f"\n[BACKTESTING GW {gw_id}] Insufficient history to train model. Skipping training/MAE calculation.")
        
    # Append the actual GW results to the full training list for the final model
    full_training_data_list.append(predict_gw_df)
        
# 3. Final Prediction for GW_TEST
print(f"\n[FINAL PREDICTION] Training model on ALL history (GW 1 to {max_backtest_gw}) and predicting GW {GW_TEST}...")

# 3a. Assemble full training data
if full_training_data_list:
    train_data_final = pd.concat(full_training_data_list, ignore_index=True)
else:
    train_data_final = pd.DataFrame()

# 3b. Train and Predict for GW_TEST
global test_df
test_df = train_and_predict(train_data_final, test_df_base)

if not test_df.empty:
    print(f"\nFinal prediction complete for GW {GW_TEST}. Top 5 predicted scorers:")
    print(test_df.sort_values(by='predicted_points_rf', ascending=False)[['web_name', 'team_name', 'position', 'predicted_points_rf']].head(5))
else:
    print("ERROR: Final prediction failed. Check training data availability.")
    
print("\nPrediction complete. Proceed to Transfer Optimization (Cell 4).")


--- Cell 3: Running Random Forest Backtesting & Prediction ---
  -> Gathering actual scores for GW 1...

[BACKTESTING GW 1] Insufficient history to train model. Skipping training/MAE calculation.
  -> Gathering actual scores for GW 1...
  -> Gathering actual scores for GW 2...

[BACKTESTING GW 2] Training model on 281 samples. Predicting GW 2...
  Actual Top Scorer: J.Timber (24.0 Pts)
  Predicted Top Scorer: Ballard (Predicted: 11.56 Pts, Actual: 1.0 Pts)
  -> Gathering actual scores for GW 1...
  -> Gathering actual scores for GW 2...
  -> Gathering actual scores for GW 3...

[BACKTESTING GW 3] Training model on 572 samples. Predicting GW 3...
  Actual Top Scorer: Guéhi (15.0 Pts)
  Predicted Top Scorer: J.Timber (Predicted: 13.79 Pts, Actual: 2.0 Pts)
  -> Gathering actual scores for GW 1...
  -> Gathering actual scores for GW 2...
  -> Gathering actual scores for GW 3...
  -> Gathering actual scores for GW 4...

[BACKTESTING GW 4] Training model on 863 samples. Predicting GW 4...
 

In [38]:
import pandas as pd
import numpy as np

# --- Global Variables used from previous cells ---
# test_df: DataFrame with all players and 'predicted_points_rf' for GW_TEST
# simulated_current_team_list: List of player IDs in the current squad
# SIM_BANK: Current bank (in 100k units)
# SIM_FT: Number of free transfers available (e.g., 1 or 2)
# pos_map: Dictionary mapping position_id to position string

# --- Position/Formation Constraints ---
# Minimum required players for a valid starting XI in each position
MIN_POS_START = {
    'GKP': 1,
    'DEF': 3,
    'MID': 3,
    'FWD': 1
}

# Total required players for a valid starting XI
REQUIRED_STARTERS = sum(MIN_POS_START.values()) # Should be 8, plus 3 flex spots = 11

# --- Utility Functions ---

def get_current_squad_df(test_df, current_squad_ids):
    """Filters the test_df to show only players currently in the squad."""
    current_squad_df = test_df[test_df['id'].isin(current_squad_ids)].copy()
    current_squad_df['is_current'] = True
    return current_squad_df

def calculate_best_xi_score(squad_df):
    """
    Calculates the maximum predicted points for a valid starting XI from the given squad.
    This is a constrained maximization problem, simplified using sorting and selection.
    """
    if squad_df.empty or len(squad_df) < 11:
        return 0, []

    squad_df = squad_df.sort_values(by='predicted_points_rf', ascending=False)
    
    # 1. Select the required minimum players for each position
    starting_xi_ids = []
    points = 0
    
    remaining_squad = squad_df.copy()
    
    for pos, min_count in MIN_POS_START.items():
        # Get top 'min_count' players for this position
        top_pos_players = remaining_squad[remaining_squad['position'] == pos].head(min_count)
        
        starting_xi_ids.extend(top_pos_players['id'].tolist())
        points += top_pos_players['predicted_points_rf'].sum()
        
        # Remove selected players from remaining pool
        remaining_squad = remaining_squad.drop(top_pos_players.index)

    # 2. Select remaining players to fill the XI (total 11) from the rest of the squad
    remaining_needed = 11 - len(starting_xi_ids)
    
    if remaining_needed > 0:
        # Sort the remainder by predicted points and take the best
        flex_players = remaining_squad.sort_values(by='predicted_points_rf', ascending=False).head(remaining_needed)
        
        starting_xi_ids.extend(flex_players['id'].tolist())
        points += flex_players['predicted_points_rf'].sum()
        
    return points, starting_xi_ids

# ----------------------------------------------------------------------
# --- OPTIMIZATION EXECUTION ---
# ----------------------------------------------------------------------

print("--- Cell 4: Transfer Optimization for GW {} ---".format(GW_TEST))

# 1. Prepare Squad Data
current_squad_df = get_current_squad_df(test_df, simulated_current_team_list)
market_df = test_df[~test_df['id'].isin(simulated_current_team_list)].copy()

# Initial Score
initial_score, _ = calculate_best_xi_score(current_squad_df)
print(f"Current Team Predicted Best XI Score for GW {GW_TEST}: {initial_score:.2f} Pts")

transfer_suggestions = [] # List to hold all potential transfers

# 2. Optimization Loop (Iterate through possible 1-for-1 swaps)
if SIM_FT < 1:
     print("\nWARNING: No free transfers available (SIM_FT=0). All moves will incur a -4 point hit.")

print(f"\nSearching for optimal 1-transfer moves (FTs available: {SIM_FT})...")

# Iterate through all players currently IN the squad (SELL candidates)
for index_out, player_out in current_squad_df.iterrows():
    
    sell_price = player_out['price_100k']
    new_bank_after_sell = SIM_BANK + sell_price

    # Iterate through all players NOT in the squad (BUY candidates)
    # Must be the same position as the player being sold
    for index_in, player_in in market_df[market_df['position'] == player_out['position']].iterrows():
        
        buy_price = player_in['price_100k']
        
        # 2a. Check Budget Constraint
        if buy_price > new_bank_after_sell:
            continue
            
        # 2b. Check Team Slot Constraint (Max 3 players per team)
        current_team_count = current_squad_df[current_squad_df['team_id'] == player_in['team_id']].shape[0]
        
        if player_in['team_id'] != player_out['team_id'] and current_team_count >= 3:
            continue

        # 2c. Simulate the new squad
        simulated_new_squad_ids = current_squad_df['id'].tolist()
        
        try:
            simulated_new_squad_ids.remove(player_out['id'])
            simulated_new_squad_ids.append(player_in['id'])
        except ValueError:
            continue # Skip if player_out['id'] somehow wasn't in the list

        simulated_squad_df = get_current_squad_df(test_df, simulated_new_squad_ids)

        # Calculate the new predicted score for the best XI
        new_score, _ = calculate_best_xi_score(simulated_squad_df)
        
        gross_gain = new_score - initial_score
        
        # 2d. Calculate Costs and Net Gain
        # We assume this is the *first* transfer the user makes this week
        point_hit = 0 if SIM_FT >= 1 else 4
        
        # A single transfer always uses 1 FT. If SIM_FT=0, it costs -4 points.
        net_gain = gross_gain - point_hit

        # 2e. Record all good transfers (positive net gain, or close to zero)
        if net_gain >= -2.0: # Keep transfers that break even or lose only 2 points, just in case
            transfer_suggestions.append({
                'OUT': player_out['web_name'],
                'IN': player_in['web_name'],
                'POSITION': player_out['position'],
                'COST_M': buy_price / 10,
                'PRICE_OUT_M': sell_price / 10,
                'GROSS_GAIN': gross_gain,
                'POINT_HIT': point_hit,
                'NET_GAIN': net_gain,
                'NEW_SCORE': new_score,
                'NEW_BANK_M': (new_bank_after_sell - buy_price) / 10
            })

# 3. Output Results

if not transfer_suggestions:
    print("\n✅ NO BENEFICIAL TRANSFERS FOUND (Net Gain < -2.0 Pts).")
    print("The current squad is likely optimal or marginal gains are too small to justify a move.")
else:
    # Sort by NET_GAIN descending to prioritize the best moves
    suggestions_df = pd.DataFrame(transfer_suggestions).sort_values(by='NET_GAIN', ascending=False).head(5)
    
    print(f"\n✅ TOP 5 TRANSFER OPTIONS FOR GW {GW_TEST} (Ranked by Net Score Gain):")
    print("-" * 75)
    
    for _, tx in suggestions_df.iterrows():
        hit_str = f"(-{tx['POINT_HIT']} Pts Hit)" if tx['POINT_HIT'] > 0 else "(Free Transfer)"
        
        print(f"| {tx['OUT']} OUT (£{tx['PRICE_OUT_M']:.1f}m) -> {tx['IN']} IN (£{tx['COST_M']:.1f}m) | Pos: {tx['POSITION']}")
        print(f"|   -> GROSS GAIN: +{tx['GROSS_GAIN']:.2f} Pts | NET GAIN: +{tx['NET_GAIN']:.2f} Pts {hit_str}")
        print(f"|   -> New Score: {tx['NEW_SCORE']:.2f} Pts | New Bank: £{tx['NEW_BANK_M']:.1f}m")
        print("-" * 75)

# 4. Show current squad for context
print("\n--- Current Squad Predicted Points (GW {}) ---".format(GW_TEST))
current_squad_df['price_m'] = current_squad_df['price_100k'] / 10
print(current_squad_df[['web_name', 'team_name', 'position', 'price_m', 'predicted_points_rf']]
      .sort_values(by='predicted_points_rf', ascending=False)
      .to_string(index=False))

print("\nOptimization complete. Review the recommendation before proceeding.")


--- Cell 4: Transfer Optimization for GW 8 ---
Current Team Predicted Best XI Score for GW 8: 70.04 Pts

Searching for optimal 1-transfer moves (FTs available: 1)...

✅ TOP 5 TRANSFER OPTIONS FOR GW 8 (Ranked by Net Score Gain):
---------------------------------------------------------------------------
| Tarkowski OUT (£5.5m) -> J.Timber IN (£5.9m) | Pos: DEF
|   -> GROSS GAIN: +2.47 Pts | NET GAIN: +2.47 Pts (Free Transfer)
|   -> New Score: 72.51 Pts | New Bank: £0.1m
---------------------------------------------------------------------------
| Tarkowski OUT (£5.5m) -> Chalobah IN (£5.1m) | Pos: DEF
|   -> GROSS GAIN: +2.38 Pts | NET GAIN: +2.38 Pts (Free Transfer)
|   -> New Score: 72.42 Pts | New Bank: £0.9m
---------------------------------------------------------------------------
| Tarkowski OUT (£5.5m) -> Burn IN (£5.1m) | Pos: DEF
|   -> GROSS GAIN: +2.23 Pts | NET GAIN: +2.23 Pts (Free Transfer)
|   -> New Score: 72.27 Pts | New Bank: £0.9m
----------------------------------

In [39]:
import pandas as pd
import numpy as np

# --- Global Variables used from previous cells ---
# test_df: DataFrame with all players and 'predicted_points_rf' for GW_TEST
# simulated_current_team_list: List of player IDs in the current squad
# GW_TEST: Target Gameweek (e.g., 8)

# --- Position/Formation Constraints (Re-declared for completeness) ---
MIN_POS_START = {
    'GKP': 1,
    'DEF': 3,
    'MID': 3,
    'FWD': 1
}

# --- Utility Functions (Copied from Cell 4 for independence) ---

def get_current_squad_df(test_df, current_squad_ids):
    """Filters the test_df to show only players currently in the squad."""
    current_squad_df = test_df[test_df['id'].isin(current_squad_ids)].copy()
    current_squad_df['is_current'] = True
    return current_squad_df

def calculate_best_xi_score(squad_df):
    """
    Calculates the maximum predicted points for a valid starting XI from the given squad.
    Returns the score and the list of IDs in the starting XI.
    """
    if squad_df.empty or len(squad_df) < 11:
        return 0, []

    squad_df = squad_df.sort_values(by='predicted_points_rf', ascending=False)
    
    # 1. Select the required minimum players for each position
    starting_xi_ids = []
    points = 0
    
    remaining_squad = squad_df.copy()
    
    for pos, min_count in MIN_POS_START.items():
        # Get top 'min_count' players for this position
        top_pos_players = remaining_squad[remaining_squad['position'] == pos].head(min_count)
        
        starting_xi_ids.extend(top_pos_players['id'].tolist())
        points += top_pos_players['predicted_points_rf'].sum()
        
        # Remove selected players from remaining pool
        remaining_squad = remaining_squad.drop(top_pos_players.index)

    # 2. Select remaining players to fill the XI (total 11) from the rest of the squad
    remaining_needed = 11 - len(starting_xi_ids)
    
    if remaining_needed > 0:
        # Sort the remainder by predicted points and take the best
        flex_players = remaining_squad.sort_values(by='predicted_points_rf', ascending=False).head(remaining_needed)
        
        starting_xi_ids.extend(flex_players['id'].tolist())
        points += flex_players['predicted_points_rf'].sum()
        
    return points, starting_xi_ids

# ----------------------------------------------------------------------
# --- EXECUTION ---
# ----------------------------------------------------------------------

print("--- Cell 5: Captaincy, Starting XI, and Bench Order for GW {} ---".format(GW_TEST))

# 1. Get the current squad with predicted points
current_squad_df = get_current_squad_df(test_df, simulated_current_team_list)

if current_squad_df.empty or len(current_squad_df) < 15:
    print("Error: Current squad data is missing or incomplete.")
    exit()

# 2. Determine the Optimal Starting XI
total_score, starting_xi_ids = calculate_best_xi_score(current_squad_df)

if not starting_xi_ids:
    print("Error: Could not determine a valid starting XI.")
    exit()
    
starting_xi_df = current_squad_df[current_squad_df['id'].isin(starting_xi_ids)].copy()
bench_df = current_squad_df[~current_squad_df['id'].isin(starting_xi_ids)].copy()

# 3. Captaincy Selection (from the Starting XI)
captaincy_df = starting_xi_df.sort_values(by='predicted_points_rf', ascending=False)
captain = captaincy_df.iloc[0]
vice_captain = captaincy_df.iloc[1]

# 4. Bench Ordering (Ranked by predicted points)
bench_df = bench_df.sort_values(by='predicted_points_rf', ascending=False)
bench_players = bench_df[['web_name', 'position', 'predicted_points_rf']].reset_index(drop=True)

# 5. Output Results

print("\n--- CAPTAINCY RECOMMENDATION ---")
print("-" * 40)
print(f"🥇 CAPTAIN (C): {captain['web_name']} ({captain['team_name']})")
print(f"   -> Predicted Points: {captain['predicted_points_rf']:.2f} (x2 = {captain['predicted_points_rf'] * 2:.2f})")
print(f"🥈 VICE-CAPTAIN (VC): {vice_captain['web_name']} ({vice_captain['team_name']})")
print(f"   -> Predicted Points: {vice_captain['predicted_points_rf']:.2f}")
print("-" * 40)

print("\n--- OPTIMAL STARTING XI (11 Players) ---")
xi_display = starting_xi_df[['web_name', 'team_name', 'position', 'predicted_points_rf']]
xi_display = xi_display.sort_values(by='predicted_points_rf', ascending=False)

# Add (C) and (VC) flags for display
xi_display['Player'] = xi_display.apply(
    lambda row: f"{row['web_name']} (C)" if row['web_name'] == captain['web_name']
    else (f"{row['web_name']} (VC)" if row['web_name'] == vice_captain['web_name'] else row['web_name']), 
    axis=1
)

print(xi_display[['Player', 'team_name', 'position', 'predicted_points_rf']]
      .rename(columns={'predicted_points_rf': 'Pts_Pred'})
      .to_string(index=False))

print(f"\nTotal Predicted Starting XI Score (excluding captaincy bonus): {total_score:.2f} Pts")

print("\n--- OPTIMAL BENCH ORDER (Sub 1, Sub 2, Sub 3) ---")
print("Order bench players by highest predicted points for best auto-sub coverage.")

if not bench_players.empty:
    for i in range(len(bench_players)):
        player = bench_players.iloc[i]
        print(f"Sub {i+1}: {player['web_name']} ({player['position']}) - {player['predicted_points_rf']:.2f} Pts")
else:
    print("Bench players list is empty.")

print("\nFinal analysis complete.")


--- Cell 5: Captaincy, Starting XI, and Bench Order for GW 8 ---

--- CAPTAINCY RECOMMENDATION ---
----------------------------------------
🥇 CAPTAIN (C): Haaland (Man City)
   -> Predicted Points: 9.93 (x2 = 19.86)
🥈 VICE-CAPTAIN (VC): Semenyo (Bournemouth)
   -> Predicted Points: 9.71
----------------------------------------

--- OPTIMAL STARTING XI (11 Players) ---
      Player      team_name position  Pts_Pred
 Haaland (C)       Man City      FWD      9.93
Semenyo (VC)    Bournemouth      MID      9.71
     Gabriel        Arsenal      DEF      6.76
      Senesi    Bournemouth      DEF      6.59
        Pope      Newcastle      GKP      5.92
     Anthony        Burnley      MID      5.87
  João Pedro        Chelsea      FWD      5.78
   Cucurella        Chelsea      DEF      5.21
     Vicario          Spurs      GKP      4.90
    Richards Crystal Palace      DEF      4.87
       Gakpo      Liverpool      MID      4.50

Total Predicted Starting XI Score (excluding captaincy bonus): 7

In [40]:
import pandas as pd
import numpy as np
import itertools

# --- Global Variables used from previous cells ---
# test_df: DataFrame with all players and 'predicted_points_rf' for GW_TEST
# simulated_current_team_list: List of player IDs in the current squad
# SIM_BANK: Current bank (in 100k units)
# SIM_FT: Number of free transfers available (e.g., 1 or 2)
# GW_TEST: Target Gameweek (e.g., 8)

# --- Position/Formation Constraints (Re-declared for completeness) ---
MIN_POS_START = {
    'GKP': 1,
    'DEF': 3,
    'MID': 3,
    'FWD': 1
}

# --- Utility Functions (Complete, based on Cell 4/5) ---

def get_current_squad_df(test_df, current_squad_ids):
    """Filters the test_df to show only players currently in the squad."""
    current_squad_df = test_df[test_df['id'].isin(current_squad_ids)].copy()
    current_squad_df['is_current'] = True
    return current_squad_df

def calculate_best_xi_score(squad_df):
    """Calculates the maximum predicted points for a valid starting XI from the given squad."""
    if squad_df.empty or len(squad_df) < 11:
        return 0, []

    squad_df = squad_df.sort_values(by='predicted_points_rf', ascending=False)
    starting_xi_ids = []
    remaining_squad = squad_df.copy()
    
    # 1. Select the required minimum players for each position
    for pos, min_count in MIN_POS_START.items():
        top_pos_players = remaining_squad[remaining_squad['position'] == pos].head(min_count)
        starting_xi_ids.extend(top_pos_players['id'].tolist())
        remaining_squad = remaining_squad.drop(top_pos_players.index)

    # 2. Select remaining players to fill the XI (total 11)
    remaining_needed = 11 - len(starting_xi_ids)
    
    if remaining_needed > 0:
        flex_players = remaining_squad.sort_values(by='predicted_points_rf', ascending=False).head(remaining_needed)
        starting_xi_ids.extend(flex_players['id'].tolist())
        
    final_xi_df = squad_df[squad_df['id'].isin(starting_xi_ids)]
    points = final_xi_df['predicted_points_rf'].sum()

    return points, starting_xi_ids

# ----------------------------------------------------------------------
# --- OPTIMIZED TWO-TRANSFER EXECUTION ---
# ----------------------------------------------------------------------

print("--- Cell 6: FAST Advanced Multi-Transfer Optimization (Two Transfers) ---")

# 1. Prepare Squad Data
current_squad_df = get_current_squad_df(test_df, simulated_current_team_list)
market_df = test_df[~test_df['id'].isin(simulated_current_team_list)].copy()

# Initial Score
initial_score, _ = calculate_best_xi_score(current_squad_df)
print(f"Current Team Predicted Best XI Score for GW {GW_TEST}: {initial_score:.2f} Pts")

two_transfer_suggestions = []
max_suggestions = 10 

# Determine the point hit for making TWO transfers
if SIM_FT >= 2:
    point_hit = 0
elif SIM_FT == 1:
    point_hit = 4
else: # SIM_FT == 0
    point_hit = 8
    
print(f"Analyzing 2-transfer moves. Point Hit: -{point_hit} Pts.")
print("-" * 50)

# 2. Heuristic Filtering (Drastically reduces search space)
# A. Identify the worst 5 players in the current squad to sell
#    We use head(6) for safety/flexibility, but the key is low numbers.
squad_sell_candidates = current_squad_df.sort_values(by='predicted_points_rf', ascending=True).head(6)

# B. Identify the best 30 players in the market to buy
market_buy_candidates = market_df.sort_values(by='predicted_points_rf', ascending=False).head(30)

current_squad_list = [row for index, row in squad_sell_candidates.iterrows()]
market_list = [row for index, row in market_buy_candidates.iterrows()]


# 3. Optimized Nested Loops
# Generate combinations of 2 players to sell (Out 1, Out 2)
for players_out_tuple in itertools.combinations(current_squad_list, 2):
    p_out1 = players_out_tuple[0]
    p_out2 = players_out_tuple[1]

    # Iterate through all possible pairs of players to buy (In 1, In 2)
    for players_in_tuple in itertools.product(market_list, repeat=2):
        p_in1 = players_in_tuple[0]
        p_in2 = players_in_tuple[1]
        
        # 1. Basic Checks
        if p_in1['id'] == p_in2['id']: continue
        
        # 2. Position Check: Must sell the same position as bought
        if p_out1['position'] != p_in1['position'] or p_out2['position'] != p_in2['position']: continue
            
        # --- Transaction Simulation ---
        sell_value = p_out1['price_100k'] + p_out2['price_100k']
        buy_cost = p_in1['price_100k'] + p_in2['price_100k']
        
        new_bank = SIM_BANK + sell_value - buy_cost
        
        # 3. Budget Check
        if new_bank < 0: continue
            
        # --- Simulated Squad Assembly ---
        sim_squad_ids = simulated_current_team_list.copy()
        
        # Remove players out
        try:
            sim_squad_ids.remove(p_out1['id'])
            sim_squad_ids.remove(p_out2['id'])
        except ValueError:
            continue # Should not happen
            
        # Add players in
        sim_squad_ids.append(p_in1['id'])
        sim_squad_ids.append(p_in2['id'])
        
        simulated_squad_df = get_current_squad_df(test_df, sim_squad_ids)
        
        # 4. Team Limit Check (Max 3 players per team)
        team_counts = simulated_squad_df['team_id'].value_counts()
        if (team_counts > 3).any(): continue
            
        # 5. Calculate Score and Net Gain
        new_score, _ = calculate_best_xi_score(simulated_squad_df)
        
        gross_gain = new_score - initial_score
        net_gain = gross_gain - point_hit

        # 6. Record good transfers
        if net_gain >= -2.0: # Only record moves that are marginally profitable or better
            two_transfer_suggestions.append({
                'OUT1': p_out1['web_name'],
                'IN1': p_in1['web_name'],
                'OUT2': p_out2['web_name'],
                'IN2': p_in2['web_name'],
                'POSITION1': p_out1['position'],
                'POSITION2': p_out2['position'],
                'GROSS_GAIN': gross_gain,
                'POINT_HIT': point_hit,
                'NET_GAIN': net_gain,
                'NEW_SCORE': new_score,
                'NEW_BANK_M': new_bank / 10
            })

# 4. Output Results

if not two_transfer_suggestions:
    print("\n✅ NO BENEFICIAL 2-TRANSFER MOVES FOUND (Net Gain < -2.0 Pts in the optimized search).")
    print("Stick to the current team or re-evaluate the single transfer options.")
else:
    # Sort by NET_GAIN descending and take the top N
    suggestions_df = pd.DataFrame(two_transfer_suggestions).sort_values(by='NET_GAIN', ascending=False).head(max_suggestions)
    
    hit_str = f"(-{point_hit} Pts Hit)" if point_hit > 0 else "(Free Transfers)"

    print(f"\n✅ TOP {len(suggestions_df)} MULTI-TRANSFER OPTIONS (2 Moves) FOR GW {GW_TEST}: {hit_str}")
    print("-" * 80)
    
    for _, tx in suggestions_df.iterrows():
        
        transfer_summary = f"({tx['OUT1']} -> {tx['IN1']}) + ({tx['OUT2']} -> {tx['IN2']})"
        
        print(f"| Transfers: {transfer_summary}")
        print(f"| Pos: {tx['POSITION1']}/{tx['POSITION2']} | GROSS GAIN: +{tx['GROSS_GAIN']:.2f} Pts | NET GAIN: +{tx['NET_GAIN']:.2f} Pts")
        print(f"| New Team Score: {tx['NEW_SCORE']:.2f} Pts | Remaining Bank: £{tx['NEW_BANK_M']:.1f}m")
        print("-" * 80)

print("\nMulti-transfer analysis complete.")

--- Cell 6: FAST Advanced Multi-Transfer Optimization (Two Transfers) ---
Current Team Predicted Best XI Score for GW 8: 70.04 Pts
Analyzing 2-transfer moves. Point Hit: -4 Pts.
--------------------------------------------------

✅ TOP 10 MULTI-TRANSFER OPTIONS (2 Moves) FOR GW 8: (-4 Pts Hit)
--------------------------------------------------------------------------------
| Transfers: (Tarkowski -> Burn) + (Richards -> Chalobah)
| Pos: DEF/DEF | GROSS GAIN: +4.58 Pts | NET GAIN: +0.58 Pts
| New Team Score: 74.62 Pts | Remaining Bank: £0.3m
--------------------------------------------------------------------------------
| Transfers: (Tarkowski -> Chalobah) + (Richards -> Burn)
| Pos: DEF/DEF | GROSS GAIN: +4.58 Pts | NET GAIN: +0.58 Pts
| New Team Score: 74.62 Pts | Remaining Bank: £0.3m
--------------------------------------------------------------------------------
| Transfers: (Gakpo -> Caicedo) + (Richards -> J.Timber)
| Pos: MID/DEF | GROSS GAIN: +4.41 Pts | NET GAIN: +0.41 Pts
| 